In [1]:
import classes
import random
from amadeus import Client, ResponseError
from hotels1 import hotelFunctions
import amadeus
#import api
import hotels1
import hotelTests
import transport
from transport import transportFunctions
from transport import getNZfromCity
import datetime

In [2]:
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
hotel = hotelFunctions(amadeus)

In [ ]:
hotels = hotel.getHotelsByGeocode(getNZfromCity('London')[0], getNZfromCity('London')[1])

In [ ]:
for i in hotels:
    print(i)

In [ ]:
transport = transportFunctions()
transports = transport.getTransport(51.5061,-0.10144,51.2191,-0.1246)


In [ ]:
for i in transports:
    print(i)

In [ ]:
attractions = hotel.getActivities(32.079664, 34.767410, 2)

In [ ]:
for i in attractions:
    print(i)

In [ ]:
def chooseCheapestHotel(hotels):
    lowest = 1000000
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost < lowest):
            index = i
            lowest = hotels[i].cost
    return hotels[index]
def chooseMostLuxuriousHotel(hotels):
    highest = 0
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost > highest):
            index = i
            highest = hotels[i].cost
    return hotels[index]

In [ ]:
lowCost = False
luxury = True
#shoomerShabat = False
tripPeople = 4
duration = 4
museumOriented = True
fastPaced = True
usedAttractions = []
maxPrice = 10000
priceCapLeft = 10000 # maybe add mostCentralLocation implementation later
amadeus = Client(
    client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
    client_secret='rKvILHDsjxcCh6yq'
)
city = "New York"
hotel = hotelFunctions(amadeus)
transport = transportFunctions()
#print(city)
#print(getNZfromCity(city))
hotels = hotel.getHotelsByGeocode(getNZfromCity(city)[0], getNZfromCity(city)[1])
for i in hotels:
    print(i)
choosenHotel = None
if (lowCost):
    choosenHotel = chooseCheapestHotel(hotels)
elif (luxury):
    choosenHotel = chooseMostLuxuriousHotel(hotels)
else:
    choosenHotel = random.choice(hotels)
print(choosenHotel)

In [ ]:
def cleanUsedAttractions(attractions, usedAttractions):
    newAttractionList = []
    for i in attractions:
        if (i.title.lower() in usedAttractions):
            continue
        newAttractionList.append(i)
    return newAttractionList

In [ ]:
transport = transportFunctions()
def planFullDay(tripPeople, luxury, lowCost, museumOriented, 
                fastPaced, priceCapLeft, attractionObject, transportObject, hotel, usedAttractions, date):
    xCoord = hotel.destination.split(',')
    yCoord = float(xCoord[1])
    xCoord = float(xCoord[0])
    print(xCoord, yCoord)
    attractions = [] # maybe add most_central_location implementation later
    while (attractions == []):
        attractions = attractionObject.getActivities(xCoord, yCoord, 10)
        attractions = cleanUsedAttractions(attractions, usedAttractions)
        noise = random.random() - 0.5 * 0.2
        xCoord += noise
        yCoord += noise
    attractionOptions = []
    for i in attractions:
        print(i)
    if (museumOriented):
        for i in attractions:
            if ('museum' in i.title.lower()):
                attractionOptions.append(i)
        if (attractionOptions == []):
            attractionOptions = attractions
            noise = random.random() - 0.5 * 0.2
            xCoord += noise
            yCoord += noise
    else:
        attractionOptions = attractions
    if (lowCost):
        attractionOptions.sort(key=lambda x: x.cost)
    elif (luxury):
        attractionOptions.sort(key=lambda x: x.cost, reverse=True)
    day, usedAttractions = planDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions)

In [ ]:
date = datetime.datetime.now()


In [ ]:
print(planFullDay(tripPeople, luxury, lowCost,
            museumOriented, fastPaced, priceCapLeft, hotel, transport, choosenHotel, usedAttractions, date))

In [40]:
transport = transportFunctions()
#print(datetime.datetime.now())
date = datetime.datetime.now()
date = date.replace(minute=59, hour=23, second=59, year=2022, month=10, day=1)
a = transport.getTransportByTime(51.5061,-0.10144,51.2191,-0.1246, str(date.isoformat()))
for i in a:
    print(i)
    pass

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T06:50:00+01:00', 'place': {'type': 'place', 'location': {'lat': 51.5061, 'lng': -0.10144}}}, 'arrival': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S0-arr', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'arrival': {'time': '2022-10-02T07:09:00+01:00', 'place': {'name': 'London Bridge', 'type': 'station', 'location': {'lat': 51.50511, 'lng': -0.08609}, 'wheelchairAccessible': 'yes', 'id': '120011305', 'code': 'LNDNBDC'}}, 'spans': [{'walkAttributes': ['indoor']}], 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-10-02

In [14]:
def planDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions):
    currDate = date.replace(minute = 0, hour = 9, second=0)
    attractionArr = []
    transportArr = []
    currX = xCoord
    currY = yCoord
    attractionCount = 2
    if (fastPaced):
        attractionCount = 3
    for i in range(0, attractionCount):
        attractionLocation = attractionOptions[i].destination.split(',')
        destX = float(attractionLocation[0])
        destY = float(attractionLocation[1])
        print(currX, currY, destX, destY, str(currDate.isoformat()))
        currTransport = transport.getTransportByTime(currX, currY, destX, destY, str(currDate.isoformat()))
        currX = destX
        currY = destY
        print(currTransport)
        currDate = currTransport[-1].timeEnd
        currDate = currDate + datetime.timedelta(minutes=5)
        transportArr.append(transport)
        attractionOptions[i].timeStart = currDate
        currDate = currDate + datetime.timedelta(hours=3)
        attractionOptions[i].timeEnd = currDate
        currDate = currDate + datetime.timedelta(minutes=5)
        attractionArr.append(attractionOptions[i])
        usedAttractions.append(attractionOptions[i].title.lower())